In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### the data 

In [ ]:
train_data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train_data[['cabin-a', 'cabin-b', 'cabin-c']] = train_data['Cabin'].str.split('/',expand=True)
test_data[['cabin-a', 'cabin-b', 'cabin-c']] = test_data['Cabin'].str.split('/',expand=True)
train_data['cabin-b']=train_data['cabin-b'].astype('float')
test_data['cabin-b']=test_data['cabin-b'].astype('float')


train_data["Transported"]=1*train_data["Transported"]

In [ ]:
X=train_data.drop("Transported",axis=1)
y=train_data["Transported"]
Xtest = test_data

In [ ]:
drop_feats=['PassengerId','Name','Cabin']
X = X.drop(drop_feats, axis=1)
Xtest =Xtest.drop(drop_feats,axis=1)

## Data Manipulation


In [ ]:
cat_cols = [o for o in X.columns if X[o].dtype=='object' and (len(X[o].unique())>0 and len(X[o].unique())<10)]
num_cols = [o for o in X.columns if X[o].dtype=='float64' or X[o].dtype=='int64']
print("cats",cat_cols)
print("nums",num_cols)

In [ ]:
from sklearn.impute import SimpleImputer

imputer1 = SimpleImputer()
X[num_cols] = pd.DataFrame(imputer1.fit_transform(X[num_cols]))
Xtest[num_cols] = pd.DataFrame(imputer1.transform(Xtest[num_cols]))

imputer2 = SimpleImputer(strategy='most_frequent')
X[cat_cols] = pd.DataFrame(imputer2.fit_transform(X[cat_cols]))
Xtest[cat_cols] = pd.DataFrame(imputer2.transform(Xtest[cat_cols]))


In [ ]:
for i in X.columns:
    print(i,"->",sum(X[i].isnull()))

In [ ]:
sum(y.isnull())

## Ordinal Encoding for categorical variables

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

X[cat_cols] = pd.DataFrame(encoder.fit_transform(X[cat_cols]))
Xtest[cat_cols] = pd.DataFrame(encoder.transform(Xtest[cat_cols]))

## training and validation split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid= train_test_split(X,y,test_size=0.2)

## correlation matrix

In [ ]:
corr_matrix =train_data.corr()
corr_matrix["Transported"].sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Scatter matrix

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(train_data,figsize=(20,10))
plt.show()

## CrossValidator


In [ ]:
def crossvalidate(model,X,y,cv,scoring):
    score= -1 * cross_val_score(model,X,y,
                          cv=cv,
                          scoring=scoring)
    print(score)
    print(score.mean())
    return score

## RandomForestRegressor

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV


### finding the best hyperparameter

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# model1= RandomForestRegressor(random_state=0)
# param_grid=[
#     {'n_estimators':[250,500,750],'max_features':[6,7,8]}
# ]
# grid = GridSearchCV(model1, param_grid,cv=5,
#                    scoring="neg_mean_squared_error",
#                    return_train_score=True)

# grid.fit(X,y)
# grid.best_params_

### the best fit

In [ ]:
model1_fin=RandomForestRegressor(max_features= 7, n_estimators= 500, random_state=0)
model1_fin.fit(X,y)

## output

In [ ]:
res = model1_fin.predict(Xtest)
res = res>0.5
res

In [ ]:

output= pd.DataFrame({'PassengerId':test_data['PassengerId'],
                     'Transported':res})
output.to_csv('submission.csv',index=False)
output